# [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/skojaku/applied-soft-comp/blob/master/notebooks/word-embedding.ipynb)

# Teaching computers how to understand words

Computers only understand numbers, not words. So to make computers understand text, we need to convert words into numbers. In this module, we will learn how to do this conversion using word embedding techniques.


In [1]:
# Uncomment the following line to install the required packages
# !pip install networkx gensim tqdm nltk bokeh faiss-cpu


## Distributional Hypothesis

One associates `cat` with `dog` because they have similar context, while `cat` is more different from `fish` than `dog` because they are in different contexts. This is the core idea of the **distributional hypothesis**.

Let's try to organize this information systematically. Imagine creating a giant table where:
- Each row represents a word
- Each column represents a document
- Each cell contains the count of how often that word appears in that document

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd

# Sample documents
documents = [
    "The cat chases mice in the garden",
    "The dog chases cats in the park",
    "Mice eat cheese in the house",
    "The cat and dog play in the garden"
]

# Create word count matrix
vectorizer = CountVectorizer()
count_matrix = vectorizer.fit_transform(documents)
words = vectorizer.get_feature_names_out()

# Display as a table
df = pd.DataFrame(count_matrix.toarray(), columns=words)
df.style.background_gradient(cmap='cividis', axis = None).set_caption("Word-Document Count Matrix")

Looking at our word-document matrix, something seems off. Words like "the" and "in" appear frequently in almost every document. Are these words really the most important for understanding document meaning? Let's see what happens if we count how often each word appears across all documents:

In [ ]:
word_counts = df.sum(axis=0)
word_counts_df = pd.DataFrame(word_counts, columns=["count"]).T
word_counts_df.style.background_gradient(cmap='cividis', axis = None).set_caption("Total appearances of each word")

We've discovered two problems with raw word counts:
1. Common words like "the" and "in" dominate the counts, but they tell us little about document content
2. Raw frequencies don't tell us how unique or informative a word is across documents

Think about it: if a word appears frequently in one document but rarely in others (like "cheese" in our example), it's probably more informative about that document's content than a word that appears equally frequently in all documents (like "the").

This realization leads us to two important questions:
1. How can we normalize word frequencies within each document to account for document length?
2. How can we adjust these frequencies to give more weight to words that are unique to specific documents?

**TF-IDF (Term Frequency-Inverse Document Frequency)** provides elegant solutions to both questions. Let's see how it works in the next section.

## The Need for Normalization

TF-IDF (Term Frequency-Inverse Document Frequency) offers our first practical glimpse into representing words as distributed patterns rather than isolated units.

The TF-IDF score for a word $t$ in document $d$ combines two components:

$\text{TF-IDF}(t,d) = \text{TF}(t,d) \times \text{IDF}(t)$

where:

$\text{TF}(t,d) = \dfrac{\text{count of term }t\text{ in document }d}{\text{total number of terms in document }d}$

$\text{IDF}(t) = \log\left(\dfrac{\text{total number of documents}}{\text{number of documents containing term }t}\right)$

Let's see this distributed representation in action:
First, let us consider a simple example with 5 documents about animals.

In [4]:
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np

# Sample documents about animals
documents = [
    "Cats chase mice in the garden",
    "Dogs chase cats in the park",
    "Mice eat cheese in the house",
    "Pets like dogs and cats need care",
    "Wild animals hunt in nature"
]

Second, we will split each document into words and create a vocabulary.
This process is called **tokenization**.

In [5]:
tokens = []
vocab = set()
for doc in documents:
    tokens_in_doc = doc.lower().split()
    tokens.append(tokens_in_doc)
    vocab.update(tokens_in_doc)

# create a dictionary that maps each word to a unique index
word_to_idx = {word: i for i, word in enumerate(vocab)}

Third, we will count how many times each word appears in each document. We begin by creating a placeholder matrix `tf_matrix` to store the counts.

In [ ]:
# Calculate term frequencies for each document
n_docs = len(documents) # number of documents
n_terms = len(vocab) # number of words

# This is a matrix of zeros
# with the number of rows equal to the number of words
# and the number of columns equal to the number of documents
tf_matrix = np.zeros((n_terms, n_docs))

print(tf_matrix)

And then we count how many times each word appears in each document.

In [ ]:
for doc_idx, tokens_in_doc in enumerate(tokens):
    for word in tokens_in_doc:
        term_idx = word_to_idx[word]
        tf_matrix[term_idx, doc_idx] += 1

for doc_idx in range(n_docs):
    tf_matrix[:, doc_idx] = tf_matrix[:, doc_idx] / np.sum(tf_matrix[:, doc_idx])

print(tf_matrix)

Fourth, we calculate the IDF for each word.
IDF is defined as the logarithm of the inverse document frequency.
Document frequency is the number of documents that contain the word.
Note that, if a word appears multiple times in the same document, it should only be counted once!

In [8]:
# Calculate IDF for each term
# let's use tf_matrix to calculate the document frequency
doc_freq = np.zeros(n_terms)

# Go through each word
for term_idx in range(n_terms):

    # For each word, go through each document
    for doc_idx in range(n_docs):
        # If the word appears in the document, increment the document frequency
        if tf_matrix[term_idx, doc_idx] > 0:
            doc_freq[term_idx] += 1

idf = np.log(n_docs / doc_freq)

Or alternatively, in a more efficient way, we can: 

In [19]:
doc_freq = np.sum(tf_matrix > 0, axis=1).reshape(-1)
idf = np.log(n_docs / doc_freq)

Next, we calculate the TF-IDF matrix. `tf_matrix` is a matrix of `n_terms` by `n_docs`, and `idf` is a vector of length `n_terms`. Remind that the tf-idf is given by

$
\text{TF-IDF}(t,d) = \text{TF}(t,d) \times \text{IDF}(t)
$

A naive way to do this (albeit not efficient) is to perform this using for loops

In [9]:
# Calculate TF-IDF matrix

tfidf_matrix = np.zeros((n_terms, n_docs))
for term_idx in range(n_terms):
    for doc_idx in range(n_docs):
        tfidf_matrix[term_idx, doc_idx] = tf_matrix[term_idx, doc_idx] * idf[term_idx]

A more efficient way is to use matrix multiplication.

In [10]:
tfidf_matrix = np.diag(idf) @ tf_matrix

where `np.diag(idf)` creates a diagonal matrix with `idf` on the diagonal.

A more efficient way is to use einsum, which is a powerful function for performing Einstein summation convention on arrays.

In [11]:
tfidf_matrix = np.einsum('ij,i->ij', tf_matrix, idf)

Now, we have the TF-IDF matrix as follows:

In [ ]:
print(tfidf_matrix)

Each row of the TF-IDF matrix is our first attempt at a distributed representation of a word.
Words that appear together in the same documents frequently will have similar rows.

One can reduce the dimensionality of the representation using dimensionality reduction techniques (e.g., PCA, SVD) while maintaining the structure of the data.
This is possible because tf-idf matrix is often low-rank in practice.

Let us apply PCA to reduce the dimensionality of the tf-idf matrix.

In [13]:
reducer = PCA(n_components=2)
xy = reducer.fit_transform(tfidf_matrix)

Let's visualize the result using Bokeh.

In [ ]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.io import push_notebook
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, HoverTool

output_notebook()

# Prepare data for Bokeh
source = ColumnDataSource(data=dict(
    x=xy[:, 0],
    y=xy[:, 1],
    text_x=xy[:, 0] + np.random.randn(n_terms) * 0.02,
    text_y=xy[:, 1] + np.random.randn(n_terms) * 0.02,
    term=list(word_to_idx.keys())
))

p = figure(title="Node Embeddings from Word2Vec", x_axis_label="X", y_axis_label="Y")

p.scatter('x', 'y', source=source, line_color="black", size = 30)

# Add labels to the points
p.text(x='text_x', y='text_y', text='term', source=source, text_font_size="10pt", text_baseline="middle", text_align="center")

show(p)

The power of TF-IDF lies in its ability to transform the distributional hypothesis into a practical mathematical framework. By representing words through their patterns of usage across documents, TF-IDF creates a distributed representation where semantic relationships emerge naturally from the data.

However, TF-IDF has its limitations. It only captures word-document relationships, missing out on the rich word-word relationships that occur within documents. This limitation led researchers to develop more sophisticated techniques like word2vec, which we'll explore in the next section.

## Word2Vec

With word2vec, words are represented as dense vectors, enabling us to explore their relationships using simple linear algebra. This is in contrast to traditional natural language processing (NLP) methods, such as bag-of-words and topic modeling, which represent words as discrete units or high-dimensional vectors.

![](https://miro.medium.com/v2/resize:fit:678/1*5F4TXdFYwqi-BWTToQPIfg.jpeg)

To showcase the effectiveness of word2vec, let’s walk through an example using the gensim library.

In [15]:
import gensim
import gensim.downloader
from gensim.models import Word2Vec

# Load pre-trained word2vec model from Google News
model = gensim.downloader.load('word2vec-google-news-300')

This may take some time to load.

Our first example is to find the words most similar to king.

In [ ]:
# Example usage
word = "king"
similar_words = model.most_similar(word)
print(f"Words most similar to '{word}':")
for similar_word, similarity in similar_words:
    print(f"{similar_word}: {similarity:.4f}")

A cool (yet controversial) application of word embeddings is analogy solving. Let us consider the following puzzle:

> man is to woman as king is to ___ ?

We can use word embeddings to solve this puzzle.

In [ ]:
# We solve the puzzle by
#
#  vec(king) - vec(man) + vec(woman)
#
# To solve this, we use the model.most_similar function, with positive words being "king" and "woman" (additive), and negative words being "man" (subtractive).
#
model.most_similar(positive=['woman', "king"], negative=['man'], topn=5)

The last example is to visualize the word embeddings.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

countries = ['Germany', 'France', 'Italy', 'Spain', 'Portugal', 'Greece']
capital_words = ['Berlin', 'Paris', 'Rome', 'Madrid', 'Lisbon', 'Athens']

# Get the word embeddings for the countries and capitals
country_embeddings = np.array([model[country] for country in countries])
capital_embeddings = np.array([model[capital] for capital in capital_words])

# Compute the PCA
pca = PCA(n_components=2)
embeddings = np.vstack([country_embeddings, capital_embeddings])
embeddings_pca = pca.fit_transform(embeddings)

# Create a DataFrame for seaborn
df = pd.DataFrame(embeddings_pca, columns=['PC1', 'PC2'])
df['Label'] = countries + capital_words
df['Type'] = ['Country'] * len(countries) + ['Capital'] * len(capital_words)

# Plot the data
plt.figure(figsize=(12, 10))

# Create a scatter plot with seaborn
scatter_plot = sns.scatterplot(data=df, x='PC1', y='PC2', hue='Type', style='Type', s=200, palette='deep', markers=['o', 's'])

# Annotate the points
for i in range(len(df)):
    plt.text(df['PC1'][i], df['PC2'][i] + 0.08, df['Label'][i], fontsize=12, ha='center', va='bottom',
             bbox=dict(facecolor='white', edgecolor='none', alpha=0.8))

# Draw arrows between countries and capitals
for i in range(len(countries)):
    plt.arrow(df['PC1'][i], df['PC2'][i], df['PC1'][i + len(countries)] - df['PC1'][i], df['PC2'][i + len(countries)] - df['PC2'][i],
              color='gray', alpha=0.6, linewidth=1.5, head_width=0.02, head_length=0.03)

plt.legend(title='Type', title_fontsize='13', fontsize='11')
plt.title('PCA of Country and Capital Word Embeddings', fontsize=16)
plt.xlabel('Principal Component 1', fontsize=14)
plt.ylabel('Principal Component 2', fontsize=14)
ax = plt.gca()
ax.set_axis_off()

### 🔥 Exercise 🔥

1. Using the word2vec model, find the 5 most similar words to “computer” and “science”. What do you observe about the semantic relationships between these words?
2. Perform the following word analogy tasks using word2vec and explain your findings:
  - man : woman :: king : ?
  - Paris : France :: Tokyo : ?
  - car : cars :: child : ?
3. Create a visualization similar to the country-capital example above but using:
  - Different professions and their typical workplaces (e.g., doctor-hospital, teacher-school)
  - Different languages and their countries (e.g., Spanish-Spain, French-France)
4. Advanced: Investigate the concept of “gender bias” in word embeddings:
  - Create a visualization similar to the country-capital example above but using the words "he-she", "man-woman", "king-queen"
  - Train the PCA on these words and use the trained PCA model to project profession words (e.g., “doctor”, “nurse”, “engineer”, “teacher”) onto these gender directions. What does this tell us about potential biases in the training data?
